In [ ]:
import sys
import os

sys.path.append(os.path.abspath('..\..'))

import yaml
import pandas as pd

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k

In [ ]:
# read config
with open('..\config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [ ]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
#dfs = prepare_data(dataframes=dfs)

In [ ]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
del df_test

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

df = df_train[df_train['rating']>=4].copy()
df = df.groupby('user_id')['item_id'].apply(list).tolist()
te = TransactionEncoder()
df = te.fit(df).transform(df)
df = pd.DataFrame(df, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True)
del df
frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
rules